In [34]:

import os
import json
import snowflake.connector
import requests
from dotenv import load_dotenv
from smolagents import CodeAgent, tool
from smolagents.agents import ActionStep
from smolagents import LiteLLMModel
import pandas as pd
from typing import Dict, Any, List, Optional
from tavily import TavilyClient

# Load environment variables
load_dotenv()

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

In [35]:
@tool
def query_snowflake(query: str) -> str:
    """
    Executes a SQL query against Snowflake and returns the result.
    
    Args:
        query (str): The SQL query to execute.
    
    Returns:
        str: The query result in text format.
    """
    try:
        # Connect to Snowflake
        conn = snowflake.connector.connect(
            user=os.getenv("SNOWFLAKE_USER"),
            password=os.getenv("SNOWFLAKE_PASSWORD"),
            account=os.getenv("SNOWFLAKE_ACCOUNT"),
            database=os.getenv("SNOWFLAKE_DATABASE"),
            warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
            schema=os.getenv("SNOWFLAKE_SCHEMA")
        )
        
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()

        cursor.close()
        conn.close()
        
        return f"Query executed successfully. Results:\n{results}"
    
    except Exception as e:
        return f"Error executing query: {str(e)}"

In [38]:
client = TavilyClient(TAVILY_API_KEY)
@tool
def web_search(query: str) -> str:
    """
    Searches the web for COVID-related information using Google's Custom Search API.
    
    Args:
        query (str): The search query, should be related to COVID-19.
    
    Returns:
        str: JSON string containing search results.
    """

    import requests

    try:
        response = client.search(
        query=query
        )
        return response
    
    except Exception as e:
        print(f"Error in web search: {str(e)}")
        # Return empty results on error
        return json.dumps({"results": []})
        
    except Exception as e:
        return f"Error performing web search: {str(e)}"
    

urls = ["https://www.nytimes.com/interactive/2021/us/covid-cases.html"]
@tool
def fetch_web_content(url: str) -> str:
    """
    Fetches content from a specific URL for detailed information.
    
    Args:
        url (str): The URL to fetch content from.
    
    Returns:
        str: The extracted content from the webpage.
    """
    try:
        response = client.extract(
            urls=urls
        )
        return response["results"][0]["raw_content"]
    
    except Exception as e:
        print(f"Error in fetch web content: {str(e)}")
        # Return empty results on error
        return f"This is mock content about COVID-19 research and data analysis."
        
    except Exception as e:
        return f"Error fetching web content: {str(e)}"

In [10]:
# @tool
# def generate_covid_report(state: str, snowflake_data: str, web_data: str) -> str:
#     """
#     Generates a comprehensive COVID-19 report combining Snowflake data and web information.
    
#     Args:
#         state (str): The state for which the report is generated.
#         snowflake_data (str): JSON string containing Snowflake query results.
#         web_data (str): JSON string containing web search results.
    
#     Returns:
#         str: A formatted report combining both data sources.
#     """
#     try:
#         # Parse JSON data
#         snowflake_results = json.loads(snowflake_data)
#         web_results = json.loads(web_data)
        
#         # Convert to pandas DataFrame for analysis
#         df = pd.DataFrame(snowflake_results)
        
#         # Generate report
#         report = f"# COVID-19 Comprehensive Report for {state}\n\n"
        
#         # Database analysis section
#         report += "## Database Analysis\n\n"
#         report += f"### Year-over-Year COVID-19 Statistics for {state}\n\n"
        
#         # Create a formatted table with the data
#         if not df.empty:
#             # Format the data table
#             table_rows = ["| Year | Cases | Deaths |", "|------|-------|--------|"]
#             for _, row in df.iterrows():
#                 year = int(row.get('YEAR', 0))
#                 cases = int(row.get('CASES', 0))
#                 deaths = int(row.get('DEATHS', 0))
#                 table_rows.append(f"| {year} | {cases:,} | {deaths:,} |")
            
#             report += "\n".join(table_rows) + "\n\n"
            
#             # Calculate totals
#             total_cases = df['CASES'].sum()
#             total_deaths = df['DEATHS'].sum()
#             report += f"**Total Cases:** {total_cases:,}\n"
#             report += f"**Total Deaths:** {total_deaths:,}\n"
            
#             # Calculate mortality rate
#             if total_cases > 0:
#                 mortality_rate = (total_deaths / total_cases) * 100
#                 report += f"**Mortality Rate:** {mortality_rate:.2f}%\n\n"
#         else:
#             report += "No data available from Snowflake database.\n\n"
        
#         # Web research section
#         report += "## Web Research Findings\n\n"
        
#         if "results" in web_results and web_results["results"]:
#             for idx, result in enumerate(web_results["results"], 1):
#                 report += f"### Source {idx}: {result.get('title', 'Untitled')}\n"
#                 report += f"URL: {result.get('url', 'No URL')}\n"
#                 report += f"Summary: {result.get('snippet', 'No summary available')}\n\n"
#         else:
#             report += "No relevant web research findings available.\n\n"
        
#         # Conclusion section
#         report += "## Analysis and Conclusions\n\n"
#         report += f"The combined analysis of Snowflake database records and web research for {state} indicates:\n\n"
        
#         # This section would be filled by the agent's analysis
#         report += "1. [Agent will analyze trends in cases and deaths over time]\n"
#         report += "2. [Agent will compare state data with national averages from web research]\n"
#         report += "3. [Agent will identify potential correlations between policy changes and case fluctuations]\n\n"
        
#         report += "## Recommendations\n\n"
#         report += "Based on the integrated analysis, the following recommendations can be made:\n\n"
#         report += "1. [Agent will provide data-driven recommendations]\n"
#         report += "2. [Agent will suggest areas for further research]\n"
        
#         return report
        
#     except Exception as e:
#         return f"Error generating report: {str(e)}"

In [ ]:
model_id = "xai/grok-2-1212"
model = LiteLLMModel(model_id=model_id, api_key=os.getenv("XAI_API_KEY"))
parameter_state = "Illinois"  # This can be parameterized from user input

# Create the agent
agent = CodeAgent(
    tools=[query_snowflake, web_search, fetch_web_content],
    model=model,
    max_steps=15,  # Increased steps to accommodate more complex workflow
    additional_authorized_imports=['pandas', 'json', 'requests'],
    verbosity_level=2,
)

# Give urls for specific sections
urls = {"latest_covid_news": "https://www.nytimes.com/interactive/2021/us/covid-cases.html"}

In [ ]:

# Run the agent with an enhanced prompt
agent_output = agent.run(f"""
You are a COVID-19 data analyst agent tasked with generating a comprehensive report about COVID-19 in {parameter_state}.

Follow these steps to create a complete analysis:

1. First, retrieve COVID-19 data from Snowflake by executing this query to get year-over-year COVID-19 cases and deaths:
```sql
WITH state_agg AS 
(
    SELECT 
        EXTRACT(YEAR FROM date) AS year, 
        state, 
        MAX(cases) AS cases, 
        MAX(deaths) AS deaths 
    FROM COVID19_GLOBAL_DATA_ATLAS.HLS_COVID19_USA.COVID19_USA_CASES_DEATHS_BY_STATE_DAILY_NYT
    WHERE state='{parameter_state}'
    GROUP BY EXTRACT(YEAR FROM date), state
    ORDER BY state, year)
SELECT year, state,
    cases - LAG(cases, 1, 0) OVER (PARTITION BY state ORDER BY state, year) as cases,
    deaths - LAG(deaths, 1, 0) OVER (PARTITION BY state ORDER BY state, year) as deaths,    
FROM state_agg;
```

2. use the fetch web content tool to fetch the url contents for COVID-19 information and research specific to {parameter_state}, focusing on:
   - Current trends and statistics
   - Vaccination rates and effectiveness
   - Policy changes and public health measures
   - Notable research findings

3. Combine both data sources to generate a comprehensive report that includes:
   - Statistical analysis of the Snowflake data (year-by-year trends, totals, rates)
   - Insights from web research
   - Comparative analysis between database figures and current research
   - Evidence-based conclusions and recommendations

Think carefully about how to present this information in a clear, concise, and informative manner that would be useful for public health officials and policymakers.
""")

In [33]:

response = client.search(
    query="what are the latest updates in Covid\n"
)
print(response)

{'query': 'what are the latest updates in Covid\n', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'COVID-19 Map: Tracking Cases and Vaccinations in the U.S.', 'url': 'https://abcnews.go.com/Health/coronavirus-us-tracker/cases', 'content': "Current estimated ranges of COVID cases 7-day avg per 100K <1 1 to 9 10 to 24 25 to 49 50+ Coronavirus map showing the latest updates on how many cases, deaths and hospitalizations have been recorded in the U.S. Scroll over the map to see your state's case numbers and vaccination results. COVID and Vaccine data sourced from CDC and HHS Map shading of NY State COVID cases 7-day avg per 100K excludes NYC.", 'score': 0.4358301, 'raw_content': None}, {'title': 'Covid variant surge: Latest CDC guidelines on isolation and symptoms as ...', 'url': 'https://www.nbcnews.com/health/health-news/covid-isolation-guidelines-cdc-updated-positive-cases-rcna163292', 'content': "NBC News Now But people who test positive for Covid don